We want to derive an algorithm to solve an $N \times N$ linear system of equations $\mathbf{A} \mathbf{x} = \mathbf{b}$, where the matrix $\mathbf{A}$ has nonzero elements only on its diagonal and a few other subdiagonals around the main diagonal. Such a matrix is called a band-diagonal matrix and solving systems of this type can be done in $\mathcal{O}(N)$ complexity as opposed to the $\mathcal{O}(N^3)$ complexity for solving arbitrary linear systems. Band diagonal matrices tend to occurr in problems where each element in an array is connected only to a few of its immediate neighbours, such as in interpolation problems or in finite difference schemes for differential equations, so they are quite important.

To warm up, let's first consider the $5 \times 5$ tridiagonal matrix $\mathbf{A}$ with main diagonal elements $d_i$, upper subdiagonal elements $u_i$ and lower subdiagonal elements $l_i$. We write down the augmented coefficient matrix $\mathbf{M}$  which is equal to $\mathbf{A}$ with the right-hand-side vector appended as rightmost column:

In [36]:
var("d0 d1 d2 d3 d4 u0 u1 u2 u3 l0 l1 l2 l3")
var("b0 b1 b2 b3 b4")
M = matrix([[d0,u0,0,0,0,b0],[l0,d1,u1,0,0,b1],[0,l1,d2,u2,0,b2],
            [0,0,l2,d3,u3,b3],[0,0,0,l3,d4,b4]])
M

[d0 u0  0  0  0 b0]
[l0 d1 u1  0  0 b1]
[ 0 l1 d2 u2  0 b2]
[ 0  0 l2 d3 u3 b3]
[ 0  0  0 l3 d4 b4]

We will need a means of performing the 3 elementary row operations: (1) exchange of two rows, (2) multipy a row by a nonzero constant, (3) add a multiple of one row to another row. These operations are implemented in sage as:  
`
M.swap_rows()          # interchange two rows
rescale_row()          # scale a row by using a scale factor
add_multiple_of_row()  # add a multiple of one row to another row, replacing the row
`  
as can be seen here:
http://www.cfm.brown.edu/people/dobrush/am34/sage/echelon.html


In [45]:
A = copy(M)
A.add_multiple_of_row(1,0,-A[1][0]/A[0][0]); 
A.add_multiple_of_row(2,1,-A[2][1]/A[1][1]); 
A.add_multiple_of_row(3,2,-A[3][2]/A[2][2]); 
A.add_multiple_of_row(4,3,-A[4][3]/A[3][3]); 
#A

In [47]:
A[4][5]

b4 - (b3 - (b2 - (b1 - b0*l0/d0)*l1/(d1 - l0*u0/d0))*l2/(d2 - l1*u1/(d1 - l0*u0/d0)))*l3/(d3 - l2*u2/(d2 - l1*u1/(d1 - l0*u0/d0)))

In [4]:
M = matrix([[2,1,-2,0,0,0,0,0,0],
            [1,0,1,0,0,0,0,0,0],
            [-2,1,4,1,-2,0,0,0,0],
            [0,0,1,0,1,0,0,0,0],
            [0,0,-2,1,4,1,-2,0,0],
            [0,0,0,0,1,0,1,0,0],
            [0,0,0,0,-2,1,4,1,-2],
            [0,0,0,0,0,0,1,0,1],
            [0,0,0,0,0,0,-2,1,2]])
M

[ 2  1 -2  0  0  0  0  0  0]
[ 1  0  1  0  0  0  0  0  0]
[-2  1  4  1 -2  0  0  0  0]
[ 0  0  1  0  1  0  0  0  0]
[ 0  0 -2  1  4  1 -2  0  0]
[ 0  0  0  0  1  0  1  0  0]
[ 0  0  0  0 -2  1  4  1 -2]
[ 0  0  0  0  0  0  1  0  1]
[ 0  0  0  0  0  0 -2  1  2]

In [5]:
A = copy(M)
A = A.change_ring(QQ)
A.add_multiple_of_row(1,0,-A[1][0]/A[0][0]); 
A.add_multiple_of_row(2,0,-A[2][0]/A[0][0]); 

A.add_multiple_of_row(2,1,-A[2][1]/A[1][1]); 

A.add_multiple_of_row(3,2,-A[3][2]/A[2][2]); 
A.add_multiple_of_row(4,2,-A[4][2]/A[2][2]); 

A.add_multiple_of_row(4,3,-A[4][3]/A[3][3]); 

#A = A.echelon_form()
A

[    2     1    -2     0     0     0     0     0     0]
[    0  -1/2     2     0     0     0     0     0     0]
[    0     0    10     1    -2     0     0     0     0]
[    0     0     0 -1/10   6/5     0     0     0     0]
[    0     0     0     0    18     1    -2     0     0]
[    0     0     0     0     1     0     1     0     0]
[    0     0     0     0    -2     1     4     1    -2]
[    0     0     0     0     0     0     1     0     1]
[    0     0     0     0     0     0    -2     1     2]